In [5]:
from info_titul import DatosWeb
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import urllib3


urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [ ]:
variable = DatosWeb.get_basic_data("https://www.educacion.gob.es/ruct/solicitud/detalles.action?cod=25000302008052801&sit=A&actual=menu.solicitud.basicos", str(2500030))

In [ ]:
variable

In [ ]:
variable2 = DatosWeb.get_competencies("https://www.educacion.gob.es/ruct/solicitud/competencias?actual=menu.solicitud.competencias.transversales&tipo=T&cod=25000302008052801", "2500030","Compentencias Transversales")

In [ ]:
variable2

In [ ]:
variable3 = DatosWeb.get_info("https://www.educacion.gob.es/ruct/solicitud/actividades?actual=menu.solicitud.planificacion.actividades&cod=25000302019102501", "2500030")

In [ ]:
variable3

In [ ]:
variable4 = DatosWeb.get_info("https://www.educacion.gob.es/ruct/solicitud/metodologias?actual=menu.solicitud.planificacion.metodologias&cod=25000302019102501", "2500030")

In [ ]:
variable4

In [ ]:
variable5 = DatosWeb.get_info("https://www.educacion.gob.es/ruct/solicitud/sistemas?actual=menu.solicitud.planificacion.sistemas&cod=25000302019102501", "2500030")

In [ ]:
variable5

In [ ]:
# formato base de datos
def extract_first_level(url, id):
    """
    Extracts data from the given URL and ID.
    
    Parameters:
    - url (str): The URL of the webpage.
    - id (str): The ID used in the URL.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the extracted data.
    """
    response = requests.get(url.replace('codigoin', id), verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        elementos_nivel_1_element = soup.find('input', {'id': 'modulos_tieneModulos'})
        if elementos_nivel_1_element:
            elementos_nivel_1 = elementos_nivel_1_element.get('value')
        else:
            elementos_nivel_1 = 'No encontrado'

        data = []
        table_rows = soup.select('#modulo tbody tr')
        for row in table_rows:
            denominacion = row.find_all('td')[1].get_text(strip=True)
            materia_url = row.find('a', class_='modificar')['href']
            materia_url = "https://www.educacion.gob.es" + materia_url
            materia_response = requests.get(materia_url, verify=False)
            if materia_response.status_code == 200:
                materia_soup = BeautifulSoup(materia_response.text, 'html.parser')
                materia_table_rows = materia_soup.select('#materia tbody tr')
                for materia_row in materia_table_rows:
                    asignatura = materia_row.find_all('td')[1].get_text(strip=True)
                    data.append({'Denominación': denominacion, 'Asignatura': asignatura})

        df = pd.DataFrame(data)
        return df
    else:
        print("Error: Unable to fetch data from the URL")
        return None

    
url = "https://www.educacion.gob.es/ruct/solicitud/modulos?actual=menu.solicitud.planificacion.modulos&cod=25000302019102501"
id = "2500030"
df = extract_first_level(url, id)

df

In [24]:
def extract_first_level(url, id):
    """
    Extracts data from the given URL and ID.
    
    Parameters:
    - url (str): The URL of the webpage.
    - id (str): The ID used in the URL.
    
    Returns:
    - pd.DataFrame: A DataFrame containing the extracted data.
    """
    response = requests.get(url.replace('codigoin', id), verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        elementos_nivel_1_element = soup.find('input', {'id': 'modulos_tieneModulos'})
        if elementos_nivel_1_element:
            elementos_nivel_1 = elementos_nivel_1_element.get('value')
        else:
            elementos_nivel_1 = 'No encontrado'

        data = []
        table_rows = soup.select('#modulo tbody tr')
        for row in table_rows:
            denominacion = row.find_all('td')[1].get_text(strip=True)
            materia_url = row.find('a', class_='modificar')['href']
            materia_url = "https://www.educacion.gob.es" + materia_url
            materia_response = requests.get(materia_url, verify=False)
            if materia_response.status_code == 200:
                materia_soup = BeautifulSoup(materia_response.text, 'html.parser')
                materia_table_rows = materia_soup.select('#materia tbody tr')
                for materia_row in materia_table_rows:
                    asignatura = materia_row.find_all('td')[1].get_text(strip=True)
                    asignatura_url = "https://www.educacion.gob.es" + materia_row.find('a')['href']
                    asignatura_response = requests.get(asignatura_url, verify=False)
                    data.append({'Denominación': denominacion, 'Asignatura': asignatura, 'URL': asignatura_url})

        df = pd.DataFrame(data)
        return df
    else:
        print("Error: Unable to fetch data from the URL")
        return None

url = "https://www.educacion.gob.es/ruct/solicitud/modulos?actual=menu.solicitud.planificacion.modulos&cod=25000302019102501"
id = "2500030"
df = extract_first_level(url, id)

df


,Denominación,Asignatura,URL
0,Formación básica en ciencias sociales/\ Basic ...,Introducción a la administración de empresas /...,https://www.educacion.gob.es/ruct/solicitud/da...
1,Formación básica en ciencias sociales/\ Basic ...,Introducción a la economía / Introduction to e...,https://www.educacion.gob.es/ruct/solicitud/da...
2,Formación básica en ciencias sociales/\ Basic ...,Introducción a la estadística / Introduction t...,https://www.educacion.gob.es/ruct/solicitud/da...
3,Formación básica en ciencias sociales/\ Basic ...,Introducción al derecho / Introduction to Law,https://www.educacion.gob.es/ruct/solicitud/da...
4,Formación básica en ciencias sociales/\ Basic ...,Historia económica / Economic history,https://www.educacion.gob.es/ruct/solicitud/da...
5,Formación básica en ciencias sociales/\ Basic ...,Destrezas y habilidades/ Skills and abilities,https://www.educacion.gob.es/ruct/solicitud/da...


In [35]:
import time

def extract_first_level(url, id):
    response = requests.get(url.replace('codigoin', id), verify=False)
    
    # ¿hay módulos de primer nivel?
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        elementos_nivel_1_element = soup.find('input', {'id': 'modulos_tieneModulos'})
        if elementos_nivel_1_element:
            elementos_nivel_1 = elementos_nivel_1_element.get('value')
        else:
            elementos_nivel_1 = 'No encontrado'

        table_rows = soup.select('#modulo tbody tr')
        for row in table_rows:
            denominacion = row.find_all('td')[1].get_text(strip=True)
            # página con los módulos de cada denominación
            modulo_url = row.find('a', class_='modificar')['href']
            modulo_url = "https://www.educacion.gob.es" + modulo_url
            print(modulo_url)
            time.sleep(5)  
            modulo_response = requests.get(modulo_url, verify=False)
            
            # sacamos la info del módulo
            if modulo_response.status_code == 200:
                modulo_soup = BeautifulSoup(modulo_response.text, 'html.parser')
                modulo_table_rows = modulo_soup.select('#materia tbody tr')
                
                for modulo_row in modulo_table_rows:
                    asignatura = modulo_row.find_all('td')[1].get_text(strip=True)
                    # sacamos las 
                    asignatura_url = "https://www.educacion.gob.es" + modulo_row.find('a')['href']
                    print(asignatura_url)
                    """
                    asignatura_response = requests.get(asignatura_url, verify=False)
                    asignatura_soup = BeautifulSoup(asignatura_response.text, 'html.parser')

                    # datos modulo: carácter, ects, unidad temporal
                    datos_materia_basicos_div = asignatura_soup.find('div', {'id': 'datosMateria_basicos'})
                    if datos_materia_basicos_div:
                        
                        ects_materia_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsMateria'})
                        if ects_materia_input:
                                ects_basica = ects_materia_input['value']
                                ects_obligatoria = 0
                                ects_optativa = 0
                        else:
                                ects_basica_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsBasica'})
                                ects_obligatoria_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsObligatoria'})
                                ects_optativa_input = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_ectsOptativa'})
                                ects_basica = ects_basica_input['value'] if ects_basica_input else 0
                                ects_obligatoria = ects_obligatoria_input['value'] if ects_obligatoria_input else 0
                                ects_optativa = ects_optativa_input['value'] if ects_optativa_input else 0
                            
                        caracter_element = datos_materia_basicos_div.find('input', {'id': 'datosMateria_datosBasicos_caracter_codigo'})
                        caracter = caracter_element.get('value','No encontrado')

                        unidad_temporal_element = datos_materia_basicos_div.find('input', {'id': 'datosMateria_unidadTemporal'})
                        unidad_temporal = unidad_temporal_element.get('value', 'No encontrado')
                        
                        lenguas_table = asignatura_soup.find('table', {'id': 'lengua'})
                        if lenguas_table:
                            lenguas = [lenguas_td.get_text(strip=True) for lenguas_td in lenguas_table.select('tbody td')]
                            lenguas_str = ', '.join(lenguas)
                        else:
                            lenguas_str = 'No encontrado'
                            
                        asignaturas_table = asignatura_soup.find('table', {'id': 'asignatura'})
                        if asignaturas_table:
                            asignaturas_rows = asignaturas_table.select('tbody tr')
                            asignaturas_data = []
                            for row in asignaturas_rows:
                                asignatura_id = row.find_all('td')[0].get_text(strip=True)
                                asignatura_nombre = row.find_all('td')[1].get_text(strip=True)
                                print(asignatura_nombre)
                                asignatura_url = "https://www.educacion.gob.es" + row.find('a', class_='modificar')['href']
                                asignaturas_data.append((asignatura_id, asignatura_nombre, asignatura_url))
                            print(asignaturas_data)
                        else:
                            asignaturas_str = 'No encontrado'
                  """      

        return 
        
    else:
        print("Error: Unable to fetch data from the URL")
        return None


In [36]:
url = "https://www.educacion.gob.es/ruct/solicitud/modulos?actual=menu.solicitud.planificacion.modulos&cod=25000302019102501"
id = "2500030"
df = extract_first_level(url, id)

df

https://www.educacion.gob.es/ruct/solicitud/datosModulo!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=1&actual=menu.solicitud.planificacion.materias
https://www.educacion.gob.es/ruct/solicitud/datosModulo!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=2&actual=menu.solicitud.planificacion.materias
https://www.educacion.gob.es/ruct/solicitud/datosModulo!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=3&actual=menu.solicitud.planificacion.materias
https://www.educacion.gob.es/ruct/solicitud/datosMateria!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=3&codMateria=1&actual=menu.solicitud.planificacion.materias.datos
https://www.educacion.gob.es/ruct/solicitud/datosMateria!consulta.action;jsessionid=B27802799C5BE2669114153FB08917E4?codModulo=3&codMateria=2&actual=menu.solicitud.planificacion.materias.datos
https://www.educacion.gob.es/ruct/solicitud/datosMateria!consulta.action;jsessionid=B27802799C5